In [5]:
# 1 - Install des dependances necessaires : 
%pip install -r requirements.txt


  Using cached uv-0.9.7-py3-none-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (11 kB)
Using cached uv-0.9.7-py3-none-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (21.4 MB)
Note: you may need to restart the kernel to use updated packages.


In [17]:
# 2 - Chargements des modules 
import os
from pathlib import Path
from typing import Optional, Union, Dict
import json
import boto3
import io
from urllib.parse import urlparse
from IPython.display import Markdown, display
from datetime import datetime

In [5]:


def _is_s3_uri(path: Union[str, Path]) -> bool:
    return str(path).lower().startswith("s3://")

def _split_s3_uri(s3_uri: str):
    # s3://bucket/key/with/any chars
    parsed = urlparse(s3_uri)
    bucket = parsed.netloc
    key = parsed.path.lstrip("/")
    if not bucket or not key:
        raise ValueError(f"URI S3 invalide: {s3_uri}")
    return bucket, key

def _read_s3_bytes(s3_uri: str) -> bytes:
    s3 = boto3.client("s3")
    bucket, key = _split_s3_uri(s3_uri)
    obj = s3.get_object(Bucket=bucket, Key=key)
    return obj["Body"].read()

def extract_text_from_any_file(file_path: Union[str, Path]) -> Optional[str]:
    """
    Extrait le texte de différents types de fichiers (local OU S3).
    - Texte: txt, md, log, csv, json, xml, html, py, js, java, cpp
    - PDF: PyPDF2
    - DOCX: python-docx
    - Excel: pandas (xlsx/xls)
    - HTML: BeautifulSoup
    - Images: pytesseract (si tesseract installé) — sinon voir AWS Textract ci-dessous
    """
    file_path = str(file_path)
    is_s3 = _is_s3_uri(file_path)
    extension = Path(urlparse(file_path).path if is_s3 else file_path).suffix.lower()

    # Charge le contenu en mémoire (bytes ou texte) selon la source
    raw_bytes = None
    raw_text = None

    try:
        # ---- Lecture source (S3 ou local) ----
        if is_s3:
            raw_bytes = _read_s3_bytes(file_path)
        else:
            p = Path(file_path)
            if not p.exists():
                print(f"Erreur: Le fichier {file_path} n'existe pas")
                return None
            # Pour les formats texte simples on peut lire directement en str
            if extension in ['.txt', '.md', '.log', '.csv', '.json', '.xml', '.html', '.htm', '.py', '.js', '.java', '.cpp']:
                raw_text = p.read_text(encoding='utf-8', errors='ignore')
            else:
                raw_bytes = p.read_bytes()

        # ---- Dispatch par extension ----

        # Fichiers texte simples
        if extension in ['.txt', '.md', '.log', '.py', '.js', '.java', '.cpp']:
            if raw_text is None:
                raw_text = raw_bytes.decode('utf-8', errors='ignore')
            return raw_text

        # CSV
        elif extension == '.csv':
            # si juste “extraire du texte”, on renvoie tel quel
            if raw_text is None:
                raw_text = raw_bytes.decode('utf-8', errors='ignore')
            return raw_text

        # JSON
        elif extension == '.json':
            if raw_text is None:
                raw_text = raw_bytes.decode('utf-8', errors='ignore')
            # jolisage optionnel
            try:
                return json.dumps(json.loads(raw_text), ensure_ascii=False, indent=2)
            except Exception:
                return raw_text

        # XML
        elif extension == '.xml':
            if raw_text is None:
                raw_text = raw_bytes.decode('utf-8', errors='ignore')
            return raw_text

        # HTML
        elif extension in ['.html', '.htm']:
            from bs4 import BeautifulSoup
            if raw_text is None:
                raw_text = raw_bytes.decode('utf-8', errors='ignore')
            soup = BeautifulSoup(raw_text, 'html.parser')
            return soup.get_text(separator='\n', strip=True)

        # PDF
        elif extension == '.pdf':
            import PyPDF2
            f = io.BytesIO(raw_bytes)
            reader = PyPDF2.PdfReader(f)
            pages = []
            for page in reader.pages:
                pages.append(page.extract_text() or "")
            return "\n".join(pages).strip()

        # Word (.docx)
        elif extension == '.docx':
            from docx import Document
            f = io.BytesIO(raw_bytes)
            doc = Document(f)
            return "\n".join(para.text for para in doc.paragraphs)

        # Excel (.xlsx, .xls)
        elif extension in ['.xlsx', '.xls']:
            import pandas as pd
            f = io.BytesIO(raw_bytes)
            sheets = pd.read_excel(f, sheet_name=None)
            text = []
            for sheet_name, df in sheets.items():
                text.append(f"=== Sheet: {sheet_name} ===")
                text.append(df.to_string(index=False))
            return "\n\n".join(text)

        # Images (OCR local via pytesseract)
        elif extension in ['.png', '.jpg', '.jpeg', '.tiff', '.bmp']:
            # ⚠️ nécessite tesseract installé dans l’environnement
            from PIL import Image
            import pytesseract
            img = Image.open(io.BytesIO(raw_bytes))
            return pytesseract.image_to_string(img)

        # Autres formats: tentative texte brut
        else:
            if raw_text is None and raw_bytes is not None:
                try:
                    raw_text = raw_bytes.decode('utf-8', errors='ignore')
                except Exception:
                    raw_text = None
            if raw_text:
                print(f"Avertissement: {extension} non spécifiquement supporté, retour texte brut.")
                return raw_text
            print(f"Avertissement: Type de fichier {extension} non supporté.")
            return None

    except Exception as e:
        print(f"Erreur lors de l'extraction du texte de {file_path}: {e}")
        return None


In [6]:
# 3 - fomction permettant de detecter la langue : 
def detect_language(text: str) -> str:
    """
    Détecte la langue d'un texte.
    
    Args:
        text: Texte à analyser
        
    Returns:
        Code de langue (ex: 'en', 'fr', 'es') ou message d'erreur
    """
    if not text or len(text.strip()) < 10:
        return "Erreur: Texte trop court ou vide"
    
    try:
        from langdetect import detect
        return detect(text)
    except ImportError:
        return "Erreur: La bibliothèque langdetect n'est pas installée"
    except Exception as e:
        return f"Erreur detection de langue : {str(e)}"

In [33]:
# 4 - Traduction automatique du texte en anglais : 
def translate_and_structure_bedrock(text: str, source_lang: str) -> str:
    """Traduit + structure en Markdown avec Bedrock (1 appel optimisé)."""
    
    client = boto3.client('bedrock-runtime')
    
    if source_lang == 'en':
        # Si déjà anglais, juste structurer
        prompt = f"""Structure this English regulatory document into clean Markdown.

Rules: Clear headings (# ## ###), lists, preserve all data, remove formatting artifacts.

Document:
{text}

Return ONLY Markdown:"""
    
    else:
        # Traduction en anglais + structuration
        prompt = f"""Translate this document to English AND structure it as Markdown.

Rules:
- Accurate translation to English (preserve dates, numbers, names)
- Clean Markdown structure (# ## ###, lists)
- Full content, no summary
- Remove formatting artifacts

Document:
{text}

Return ONLY English Markdown:"""
    # Limite le texte à ~400k caractères pour rester dans les limites
    truncated_text = text[:1000]
    final_prompt = prompt.replace("{text}", truncated_text)

    response = client.invoke_model(
        modelId='global.anthropic.claude-sonnet-4-5-20250929-v1:0',
        body=json.dumps({
            "anthropic_version": "bedrock-2023-05-31",
            "max_tokens": 100000,
            "messages": [{"role": "user", "content": prompt}],
            "temperature": 0.1
        })
    )
    
    result = json.loads(response['body'].read())
    return result['content'][0]['text']

In [36]:
# 4 - Traduction automatique du texte en anglais : 
def translate_and_structure_bedrock_aws(text: str, source_lang: str) -> str:
    """Traduit + structure en Markdown avec Bedrock (1 appel optimisé)."""
    
    client = boto3.client('bedrock-runtime')
    
    if source_lang == 'en':
        # Si déjà anglais, juste structurer
        prompt = f"""Structure this English regulatory document into clean Markdown.

Rules: Clear headings (# ## ###), lists, preserve all data, remove formatting artifacts.

Document:
{text}

Return ONLY Markdown:"""
    
    else:
        # Traduction en anglais + structuration
        prompt = f"""Translate this document to English AND structure it as Markdown.

Rules:
- Accurate translation to English (preserve dates, numbers, names)
- Clean Markdown structure (# ## ###, lists)
- Full content, no summary
- Remove formatting artifacts

Document:
{text}

Return ONLY English Markdown:"""
    # Limite le texte à ~400k caractères pour rester dans les limites
    truncated_text = text[:1000]
    final_prompt = prompt.replace("{text}", truncated_text)

    response = client.converse(
        modelId='global.anthropic.claude-sonnet-4-5-20250929-v1:0',
          messages=[
        {
            "role": "user",
            "content": [{"text": prompt}]
        }
    ],
    inferenceConfig={
        "temperature": 0.1,
    }
    )
    
    result = response["output"]["message"]["content"][0]["text"]
    return result





In [37]:
filepath = 's3://csv-file-store-6d12f630/dzd-4giwkqu9tkh07k/alsdvbccweepsg/shared/directives/1.DIRECTIVE (UE) 20192161 DU PARLEMENT EUROPÉEN ET DU CONSEIL.html'
extract_text = extract_text_from_any_file(filepath)
# text_in_english = translate_and_structure_bedrock(extract_text, detect_language(extract_text))
# display(Markdown(text_in_english))



In [39]:
# text_in_english = translate_and_structure_bedrock_aws(extract_text, detect_language(extract_text))
# display(Markdown(extract_text))

In [56]:
class BedrockAIClient:
    def __init__(self, model_id: str = "global.anthropic.claude-sonnet-4-5-20250929-v1:0"):
        self.bedrock_client = boto3.client('bedrock-runtime')
        self.model_id = model_id
        print(f"Using model: {self.model_id}")
    
    def generate_response(self, prompt: str, max_tokens: int = 10000) -> str:
        try:
            body = {
                "anthropic_version": "bedrock-2023-05-31",
                "messages": [{"role": "user", "content": prompt}],
                "max_tokens": max_tokens,
                "temperature": 0.1
            }
            
            response = self.bedrock_client.invoke_model(
                modelId=self.model_id,
                body=json.dumps(body)
            )
            
            response_body = json.loads(response['body'].read())
            
            if 'choices' in response_body and len(response_body['choices']) > 0:
                print(1)
                return response_body['choices'][0]['message']['content'].strip()
            else:
                return response_body
                
        except Exception as e:
            return f"Error: Unable to generate response - {str(e)}"

ai_client = BedrockAIClient()

test_response = ai_client.generate_response('Bonjour tu vas bien ?')
print(f"Test response: {test_response['content'][0]['text']}...")

Using model: global.anthropic.claude-sonnet-4-5-20250929-v1:0
Test response: Bonjour ! Je vais bien, merci de demander ! 😊

Comment allez-vous ? Que puis-je faire pour vous aider aujourd'hui ?...


In [32]:
# Enregistre le text en MD dans S3 Bucket (peux etre utiliser pour faire le RAG)

folder_uri = "s3://csv-file-store-6d12f630/dzd-4giwkqu9tkh07k/alsdvbccweepsg/UserDataUpload/"
filename = f"rapport_{datetime.now():%Y%m%d_%H%M%S}.md"
# --- Extraire bucket et prefix à partir de l’URI ---
parsed = urlparse(folder_uri)
bucket_name = parsed.netloc
prefix = parsed.path.lstrip("/")  # "notes/projets/"

# --- Construire le chemin complet du fichier ---
file_key = f"{prefix.rstrip('/')}/{filename}"

# --- Upload vers S3 ---
s3 = boto3.client("s3")
s3.put_object(
    Bucket=bucket_name,
    Key=file_key,
    Body=text_in_english.encode,
    ContentType="text/markdown"
)

print(f"✅ Fichier ajouté à : s3://{bucket_name}/{file_key}")


✅ Fichier ajouté à : s3://csv-file-store-6d12f630/dzd-4giwkqu9tkh07k/alsdvbccweepsg/UserDataUpload/rapport_20251102_075559.md


In [28]:
print(f"{text_in_english}")
text_in_english

None


In [22]:
import boto3
import pandas as pd
import numpy as np
import logging
import sagemaker
import mlflow
import os
from datetime import datetime, timezone
from sagemaker.modules import Session
from sagemaker_studio import Project

project = Project()
session = boto3.Session()
bucket_root = project.s3.root
role = project.iam_role
s3_parts = bucket_root.replace("s3://", "").split("/")
bucket = s3_parts[0]
prefix = "/".join(s3_parts[1:])
print(f"Using Bucket: {bucket}")
print(f"Using prefix: {prefix}")
print(f"Using Role: {role}")

catalog = project.connection().catalog()
id = 0
for db in catalog.databases:
    print(f"Index {id}: {db}")
    id += 1


Using Bucket: csv-file-store-6d12f630
Using prefix: dzd-4giwkqu9tkh07k/alsdvbccweepsg/dev
Using Role: arn:aws:iam::851725179216:role/datazone_usr_role_alsdvbccweepsg_cr5d9nlcbub7o0
Index 0: Database(name='salesmarketing', catalog_id='851725179216', location_uri='s3://csv-file-store-6d12f630/dzd-4giwkqu9tkh07k/alsdvbccweepsg/dev/data/catalogs/')
